In [1]:
from import_setup import *
import time
from src.simulation.cherry_picking_model import CherryPickingModel
from config.paths import DATA_PATH
import numpy as np
from src.data_management.mongo_handler import insert_simulation_result
from config.db import MONGOCLIENT_LOCAL
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# import numpy as np
# from src.simulation.cherry_picking_model import CherryPickingModel
# import pandas as pd
# import matplotlib.pyplot as plt

# for K in [1, 2, 3, 4, 5]:
#     print(f"K = {K}")
#     lamb = 1
#     scale = 1

#     model = CherryPickingModel(
#         K=K,
#         lamb=lamb,
#         theta=2.0,
#         distribution_name="exponential",
#         dist_params={"scale": scale},  ## scale = \mu
#     )
#     result = model.solve_and_analyze(num_simulations=10_000_000, use_ultra_fast=True)
#     print(model.equilibrium_results)

In [ ]:
# K = 3
# lamb = 0.5
# theta = 2.0
# scale = 3

# model = CherryPickingModel(
#     K=K,
#     lamb=lamb,
#     theta=theta,
#     distribution_name="exponential",
#     dist_params={"scale": scale},  ## scale = \mu
# )
# result = model.solve_and_analyze(num_simulations=10_000_000)

# model.calculate_Pi_R(q=10000)




In [5]:
rerun = 1
if rerun:
    ## drop the simulation_results collection
    db = MONGOCLIENT_LOCAL["cherry_picking_trap"]
    db["simulation_results"].drop()


In [6]:
db = MONGOCLIENT_LOCAL["cherry_picking_trap"]

Ks = [i for i in range(1, 50, 1)]
for i in range(50, 1050, 50):
    Ks += [i]

theta = 2.0
for lamb in np.linspace(0, 1, 11):
    lamb = round(lamb, 1)
    for scale in [0.5, 1, 1.5, 2, 3]:
        for K in Ks:
            dic = {"lamb": lamb, "K": K, "scale": scale}
            ## check whether the result already exists
            if db["simulation_results"].find_one(
                {"lamb": lamb, "K": K, "scale": scale}
            ):
                continue
            try:
                model = CherryPickingModel(
                    K=K,
                    lamb=lamb,
                    theta=theta,
                    distribution_name="exponential",
                    dist_params={"scale": scale},  ## scale = \mu
                )
                result = model.solve_and_analyze(num_simulations=1_000_000)
                dic.update({"success": True})
                dic.update(result)
            except Exception as e:
                print(e)
                dic.update({"success": False})
                dic.update({"error": str(e)})
                continue

            result.update(
                {
                    "lamb": lamb,
                    "K": K,
                    "scale": scale,
                    "dist": "exponential",
                    "theta": theta,
                }
            )
            insert_simulation_result(db, result)

In [4]:


# # Convert results to DataFrame
# df = pd.DataFrame(results)
# # df['K'] = df['K'].astype(str)

# # Create the plot
# plt.figure(figsize=(10, 6))
# plt.plot(df["K"], df["proposer_expected_payoff"], "b-", linewidth=2, label="Proposer")
# plt.plot(df["K"], df["responder_expected_payoff"], "r-", linewidth=2, label="Responder")


# # Add grid for better readability
# plt.grid(True, linestyle="--", alpha=0.7)

# # Show the plot
# plt.show()


# # Create the plot
# plt.figure(figsize=(10, 6))
# plt.plot(df["K"], df["alpha_star"], "b-", linewidth=2, )

# # Add grid for better readability
# plt.grid(True, linestyle="--", alpha=0.7)

# # Show the plot
# plt.show()



# # Create the plot
# plt.figure(figsize=(10, 6))
# plt.plot(df["K"], df["q_star"], "b-", linewidth=2, )

# # Add grid for better readability
# plt.grid(True, linestyle="--", alpha=0.7)

# # Show the plot
# plt.show()

In [ ]:
import pandas as pd
from config.db import MONGOCLIENT_LOCAL
from src.data_management.mongo_handler import get_simulation_results_filters

db = MONGOCLIENT_LOCAL["cherry_picking_trap"]


get_simulation_results_filters(db)

# df.head()

# df.columns

# df.describe()


# df[df['proposer_threshold'] == df['proposer_threshold'].max()]
# df.info()